# Using RoBERTa with Fastai - CB Tutorial 

Source:

Dev Sharma, Using RoBERTa with fast.ai for SuperGLUE Task CB
Finetuning state of the art RoBERTa with fast.ai on Commitment Bank NLP task.

This notebook follows the tutorial @ https://medium.com/@devkosal/superglue-roberta-with-fastai-for-rte-task-c362961be957

Related article:

Using RoBERTa with fast.ai for NLP
Implementing the current state of the art NLP model in fast.ai

https://medium.com/analytics-vidhya/using-roberta-with-fastai-for-nlp-7ed3fed21f6c

In [2]:
# !pip install fastai

     |████████████████████████████████| 234 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 63.9 MB 20.1 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.3 MB 227 kB/s eta 0:00:0101
     |████████████████████████████████| 127 kB 27.5 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 46.1 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 43.0 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 4.7 MB/s  eta 0:00:01
     |████████████████████████████████| 634 kB 40.3 MB/s eta 0:00:01
     |████████████████████████████████| 182 kB 36.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.4 MB 152.5 MB/s eta 0:00:01
     |████████████████████████████████| 174 kB 26.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 25.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 74.2 MB/s eta 0:00:01
     |█████████████████████████████

In [8]:
from fastai.text import *
from fastai.metrics import *
from transformers import RobertaTokenizer
import pandas as pd
from pathlib import Path

In [4]:
# Creating a config object to store task specific information
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    task = "CB",
    testing=False,
    seed = 2019,
    roberta_model_name='roberta-base', # can also be exchanged with roberta-large 
    max_lr=1e-5,
    epochs=10,
    use_fp16=False,
    bs=4, 
    max_seq_len=256, 
    num_labels = 3,
    hidden_dropout_prob=.05,
    hidden_size=768, # 1024 for roberta-large
    start_tok = "<s>",
    end_tok = "</s>",
    mark_fields=True,
)

In [9]:
path = Path(".")
data_path = path/"data"

In [10]:
train = pd.read_json(data_path/config.task/"train.jsonl",lines=True)
val = pd.read_json(data_path/config.task/"val.jsonl",lines=True)
test = pd.read_json(data_path/config.task/"test.jsonl",lines=True)

/var/folders/tj/cfj9qmvs7150dnbbhjnzz1tw0000gq/T/ipykernel_77433/1491146896.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  train = pd.read_json(data_path/config.task/"train.jsonl",lines=True)


ValueError: Expected object or value

In [27]:
# drop the unnecessary idx column
for df in (train,val):
    if "idx" in df.columns: df.drop("idx",axis=1,inplace=True)
        
if config.testing:
    train = train[:100]
    val = val[:100]
    
print(df.shape)

(56, 3)


In [28]:
train.head()

,hypothesis,label,premise
0,the language was peeled down,entailment,It was a complex language. Not written down bu...
1,no women are allowed to take part in this ritual,entailment,"It is part of their religion, a religion I do ..."
2,Gustave was shepherded into creative retreat a...,entailment,The Paris to Rouen railway was being extended ...
3,Gustave was driven to creative retreat in Croi...,entailment,Part of it was to be compulsorily purchased. Y...
4,buying places is a hobby,entailment,"Some of them, like for instance the farm in Co..."


In [29]:
train.label.value_counts()

contradiction    119
entailment       115
neutral           16
Name: label, dtype: int64

In [30]:
feat_cols = ["premise","hypothesis"]
label_cols = "label"

## Setting Up the Tokenizer

In [31]:
class FastAiRobertaTokenizer(BaseTokenizer):
    """Wrapper around RobertaTokenizer to be compatible with fastai"""
    def __init__(self, tokenizer: RobertaTokenizer, max_seq_len: int=128, **kwargs): 
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len 
    def __call__(self, *args, **kwargs): 
        return self 
    def tokenizer(self, t:str) -> List[str]: 
        """Adds Roberta bos and eos tokens and limits the maximum sequence length""" 
        if config.mark_fields:
            sub = 2 # subtraction in totoal seq_length to be made due to adding spcl tokens
            assert "xxfld" in t
            t = t.replace("xxfld 1","") # remove the xxfld 1 special token from fastai
            # converting fastai field sep token to Roberta
            t = re.split(r'xxfld \d+', t) 
            res = []
            for i in range(len(t)-1): # loop over the number of additional fields and the Roberta sep
                res += self._pretrained_tokenizer.tokenize(t[i]) + [config.end_tok, config.end_tok]
                sub += 2 # increase our subtractions since we added more spcl tokens
            res += self._pretrained_tokenizer.tokenize(t[-1]) # add the last sequence
            return [config.start_tok] + res[:self.max_seq_len - sub] + [config.end_tok] 
        
        res = self._pretrained_tokenizer.tokenize(t)
        return [config.start_tok] + res[:self.max_seq_len - sub] + [config.end_tok]

In [32]:
# create fastai tokenizer for roberta
roberta_tok = RobertaTokenizer.from_pretrained("roberta-base")

fastai_tokenizer = Tokenizer(tok_func=FastAiRobertaTokenizer(roberta_tok, max_seq_len=config.max_seq_len), 
                             pre_rules=[], post_rules=[])

In [33]:
# create fastai vocabulary for roberta
roberta_tok.save_vocabulary(path)

with open('vocab.json', 'r') as f:
    roberta_vocab_dict = json.load(f)
    
fastai_roberta_vocab = Vocab(list(roberta_vocab_dict.keys()))

In [34]:
# Setting up pre-processors
class RobertaTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
         super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False, mark_fields=config.mark_fields)

class RobertaNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=fastai_roberta_vocab, **kwargs)


def get_roberta_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for Roberta
    We remove sos and eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original Roberta model.
    """
    return [RobertaTokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor(vocab=vocab)]

## Setting up the DataBunch

In [35]:
# Creating a Roberta specific DataBunch class
class RobertaDataBunch(TextDataBunch):
    "Create a `TextDataBunch` suitable for training Roberta"
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs:int=64, val_bs:int=None, pad_idx=1,
               pad_first=True, device:torch.device=None, no_check:bool=False, backwards:bool=False, 
               dl_tfms:Optional[Collection[Callable]]=None, **dl_kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
        return cls(*dataloaders, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

In [36]:
class RobertaTextList(TextList):
    _bunch = RobertaDataBunch
    _label_cls = TextList

In [37]:
# loading the tokenizer and vocab processors
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)

# creating our databunch 
data = ItemLists(".", RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor),
                      RobertaTextList.from_df(val, ".", cols=feat_cols, processor=processor)
                ) \
       .label_from_df(cols=label_cols, label_cls=CategoryList) \
       .add_test(RobertaTextList.from_df(test, ".", cols=feat_cols, processor=processor)) \
       .databunch(bs=config.bs,pad_first=False)

# Building the Model

In [38]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification

# defining our model architecture 
class RobertaForSequenceClassificationModel(nn.Module):
    def __init__(self,num_labels=config.num_labels):
        super(RobertaForSequenceClassificationModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaForSequenceClassification.from_pretrained(config.roberta_model_name,num_labels= self.num_labels)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        outputs = self.roberta(input_ids, token_type_ids, attention_mask)
        logits = outputs[0] 
        return logits

In [39]:
roberta_model = RobertaForSequenceClassificationModel() 

learn = Learner(data, roberta_model, metrics=[accuracy])

In [40]:
learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(config.epochs, max_lr=config.max_lr)

epoch,train_loss,valid_loss,accuracy,time
0,1.079415,1.069071,0.428571,00:13
1,0.960143,0.842565,0.678571,00:12
2,0.768365,0.631433,0.750000,00:13
3,0.477261,0.324045,0.892857,00:13
4,0.279063,0.182191,0.928571,00:13
5,0.151150,0.165908,0.964286,00:14
6,0.079247,0.116142,0.964286,00:13
7,0.043399,0.111627,0.964286,00:13
8,0.027586,0.103498,0.964286,00:12
9,0.020711,0.102435,0.964286,00:13


# Getting Predictions

In [41]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values

In [42]:
# val preds
preds, pred_values = get_preds_as_nparray(DatasetType.Valid)

In [43]:
# accuracy for valid valid
(pred_values == data.valid_ds.y.items).mean()

0.9642857142857143

In [44]:
# test preds
_, test_pred_values = get_preds_as_nparray(DatasetType.Test)